## Veri Bilimi 101
>
Proje

Genel olarak yapacağınız adımlar şunlar olacak:

- Kullanacağınız veriyi indirip, okumak
- Verinizin içindeki eksik ve kategorik değişkenler ile ilgilenip modele besleyeceğimiz hale getirmek (derste gördüklerimizin üzerine de bir şeyler yapmanız hoşumuza gider demiştik, outlier'ları olan bir veride outlier'lar ile ilgili yaptıklarınızı görmek gibi şeyler olabilir mesela)
- İlgilendiğiniz probleme göre error metriğine karar vermek (derste gördüğümüz RMSE-RMSLE gibi)
- Verinizi train-validation-test diye bölmek (burada validation ve test'in gerçek hayatı yansıtması çok önemli)
- Olabildiğince fazla model denemek ve metriğimizde en iyi yapanı seçmek

# Bir kişinin Böbrek Kanseri Olma Riski 

In [11]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense, BatchNormalization, Dropout, LSTM

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
import plotly.figure_factory as ff
import missingno as msno 

In [ ]:
import plotly.graph_objects as go
import plotly.express as px 
%matplotlib inline

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

In [ ]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [39]:
veriler = "veri4.xlsx"

In [40]:
# verilerimizi dataset e atadık
dataset =pd.read_excel(veriler)
dataset

,Sex,AgeBaseline,HistoryDiabetes,HistoryCHD,HistoryVascular,HistorySmoking,HistoryHTN,HistoryDLD,HistoryObesity,DLDmeds,...,ACEIARB,CholesterolBaseline,CreatinineBaseline,eGFRBaseline,sBPBaseline,dBPBaseline,BMIBaseline,TimeToEventMonths,EventCKD35,TIME_YEAR
0,0,64,0,0,0,0,1,1,1,1,...,0,4.08,59.0,93.3,144,87,40,98,0,8
1,0,52,0,0,0,0,1,1,1,0,...,0,6.04,52.0,105.8,148,91,45,106,0,9
2,0,56,0,0,0,0,1,1,1,1,...,0,6.04,57.0,99.8,149,86,41,88,0,7
3,0,58,0,0,0,0,0,1,1,1,...,0,5.01,65.0,90.3,116,68,32,103,0,9
4,0,63,1,0,0,0,1,1,1,1,...,1,5.01,70.0,79.7,132,63,31,105,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1,25,0,0,0,0,0,0,0,0,...,0,5.03,73.0,122.6,127,74,24,102,0,9
487,1,45,0,0,0,0,1,0,1,0,...,0,5.09,67.0,110.3,150,112,33,94,0,8
488,1,44,1,0,0,0,0,1,1,1,...,0,1.06,66.3,111.6,126,83,35,0,0,0
489,1,41,0,0,1,1,1,1,0,1,...,1,4.01,93.0,87.6,119,66,22,35,1,3


In [41]:
#verilerin bilgilerini görüntüledik
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Sex                  491 non-null    int64  
 1   AgeBaseline          491 non-null    int64  
 2   HistoryDiabetes      491 non-null    int64  
 3   HistoryCHD           491 non-null    int64  
 4   HistoryVascular      491 non-null    int64  
 5   HistorySmoking       491 non-null    int64  
 6   HistoryHTN           491 non-null    int64  
 7   HistoryDLD           491 non-null    int64  
 8   HistoryObesity       491 non-null    int64  
 9   DLDmeds              491 non-null    int64  
 10  DMmeds               491 non-null    int64  
 11  HTNmeds              491 non-null    int64  
 12  ACEIARB              491 non-null    int64  
 13  CholesterolBaseline  491 non-null    float64
 14  CreatinineBaseline   491 non-null    float64
 15  eGFRBaseline         491 non-null    flo

In [42]:
# özniteliklerine baktık
dataset.columns

Index(['Sex', 'AgeBaseline', 'HistoryDiabetes', 'HistoryCHD',
       'HistoryVascular', 'HistorySmoking', 'HistoryHTN ', 'HistoryDLD',
       'HistoryObesity', 'DLDmeds', 'DMmeds', 'HTNmeds', 'ACEIARB',
       'CholesterolBaseline', 'CreatinineBaseline', 'eGFRBaseline',
       'sBPBaseline', 'dBPBaseline', 'BMIBaseline', 'TimeToEventMonths',
       'EventCKD35', 'TIME_YEAR'],
      dtype='object')

In [43]:
#kaç öznitelik ve kaç gözlemden oluştuğuna baktık
dataset.shape

(491, 22)

In [44]:
# ilk 5 satıra baktık
dataset.head()

,Sex,AgeBaseline,HistoryDiabetes,HistoryCHD,HistoryVascular,HistorySmoking,HistoryHTN,HistoryDLD,HistoryObesity,DLDmeds,...,ACEIARB,CholesterolBaseline,CreatinineBaseline,eGFRBaseline,sBPBaseline,dBPBaseline,BMIBaseline,TimeToEventMonths,EventCKD35,TIME_YEAR
0,0,64,0,0,0,0,1,1,1,1,...,0,4.08,59.0,93.3,144,87,40,98,0,8
1,0,52,0,0,0,0,1,1,1,0,...,0,6.04,52.0,105.8,148,91,45,106,0,9
2,0,56,0,0,0,0,1,1,1,1,...,0,6.04,57.0,99.8,149,86,41,88,0,7
3,0,58,0,0,0,0,0,1,1,1,...,0,5.01,65.0,90.3,116,68,32,103,0,9
4,0,63,1,0,0,0,1,1,1,1,...,1,5.01,70.0,79.7,132,63,31,105,0,9


In [45]:
# eksik verileri kontrol ettik
dataset.isnull().sum()

Sex                    0
AgeBaseline            0
HistoryDiabetes        0
HistoryCHD             0
HistoryVascular        0
HistorySmoking         0
HistoryHTN             0
HistoryDLD             0
HistoryObesity         0
DLDmeds                0
DMmeds                 0
HTNmeds                0
ACEIARB                0
CholesterolBaseline    0
CreatinineBaseline     0
eGFRBaseline           0
sBPBaseline            0
dBPBaseline            0
BMIBaseline            0
TimeToEventMonths      0
EventCKD35             0
TIME_YEAR              0
dtype: int64

In [46]:
#veri türlerine baktık
dataset.dtypes

Sex                      int64
AgeBaseline              int64
HistoryDiabetes          int64
HistoryCHD               int64
HistoryVascular          int64
HistorySmoking           int64
HistoryHTN               int64
HistoryDLD               int64
HistoryObesity           int64
DLDmeds                  int64
DMmeds                   int64
HTNmeds                  int64
ACEIARB                  int64
CholesterolBaseline    float64
CreatinineBaseline     float64
eGFRBaseline           float64
sBPBaseline              int64
dBPBaseline              int64
BMIBaseline              int64
TimeToEventMonths        int64
EventCKD35               int64
TIME_YEAR                int64
dtype: object

In [47]:
#özniteliklerin kaç benzersiz değerer sahip olduğuna baktık
dataset.nunique()

Sex                      2
AgeBaseline             59
HistoryDiabetes          2
HistoryCHD               2
HistoryVascular          2
HistorySmoking           2
HistoryHTN               2
HistoryDLD               2
HistoryObesity           2
DLDmeds                  2
DMmeds                   2
HTNmeds                  2
ACEIARB                  2
CholesterolBaseline     55
CreatinineBaseline      89
eGFRBaseline           324
sBPBaseline             77
dBPBaseline             57
BMIBaseline             37
TimeToEventMonths       93
EventCKD35               2
TIME_YEAR               10
dtype: int64

In [48]:
#verinin temel istatilksel ayrıntılarına baktık
dataset.describe()

,Sex,AgeBaseline,HistoryDiabetes,HistoryCHD,HistoryVascular,HistorySmoking,HistoryHTN,HistoryDLD,HistoryObesity,DLDmeds,...,ACEIARB,CholesterolBaseline,CreatinineBaseline,eGFRBaseline,sBPBaseline,dBPBaseline,BMIBaseline,TimeToEventMonths,EventCKD35,TIME_YEAR
count,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,...,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000
mean,0.509165,53.203666,0.437882,0.091650,0.059063,0.152749,0.682281,0.645621,0.505092,0.551935,...,0.446029,4.474420,67.856823,98.116090,131.374745,76.871690,30.183299,83.065173,0.114053,6.940937
std,0.500426,13.821282,0.496632,0.288825,0.235983,0.360113,0.466064,0.478812,0.500484,0.497803,...,0.497586,1.263057,17.918627,18.503267,15.692650,10.711134,6.236742,26.011140,0.318200,2.201710
min,0.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.020000,6.000000,60.000000,92.000000,41.000000,13.000000,0.000000,0.000000,0.000000
25%,0.000000,44.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,4.010000,55.000000,86.400000,121.000000,69.000000,26.000000,77.000000,0.000000,6.000000
50%,1.000000,54.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,5.010000,66.000000,98.100000,131.000000,77.000000,30.000000,93.000000,0.000000,8.000000
75%,1.000000,64.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,5.070000,78.500000,109.500000,141.000000,83.000000,33.000000,100.000000,0.000000,8.000000
max,1.000000,89.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,9.030000,123.000000,242.600000,180.000000,112.000000,57.000000,111.000000,1.000000,9.000000


## Elimizdeki veri seti bize şu bilgileri sunuyor;
- Sex : cinsiyet 0: kadın, 1: erkek
- AgeBaseline : hastaların yaşı
- HistoryDiabetes : Hastanın şeker hastası olup olmadığı
- HistoryCHD : Kalp hastalığı
- HistoryVascular : Damar hastalığı
- HistorySmoking : Sigara kullanımı
- HistoryHTN : Hiper tansiyon
- HistoryDLD : Gen hastalığı
- HistoryObesity : Obezite
- DLDmeds : Enzim ilacı
- DMmeds : Diyabet ilacı
- HTNmeds : Tansiyon ilacı
- ACEIARB : Böbrek kanseri ilacı
- CholestrolBaseline : Kolestrol değeri
- CreatinineBaseline : Kretin (vücuttaki organik asit) değeri
- eGFRBaseline : eGFRB kan testi
- sBPBaseline : Kan basınç farkı
- dBPBaseline : Tansiyon basınç farkı 
- BMIBaseline : Vücut kitle indexi
- EventCKD35 :Böbrek hastalığından ölüp ölmediği 
- TIME_YEAR : Hastanın takip edildiği süre

In [49]:
import seaborn as sns

In [ ]:
#öncelikle hedefi değerlendirelim ve verilerimizin dengesiz olup olmadığını öğrenelim
cols= ["#6daa9f","#774571"]
sns.countplot(x= dataset["EventCKD35"], palette= cols)

### 1: ölenlerin sayısı
## Takip edilen hastalardaki ölüm oranının %20'nin altında olduğunu görüyoruz.

## Analizimize, hedef değişkenin bir sayım grafiğini çizerek başlıyoruz. Özelliğin önemini incelemek için çeşitli niteliklerin bir korelasyon matrisi.

In [ ]:
cmap = sns.diverging_palette(150, 218, s=80, l=55, n=9)

corrmat = dataset.corr()
plt.subplots(figsize=(18,18))
sns.heatmap(corrmat,cmap= cmap,annot=True, square=True);

### Korelasyon matrisine göre ölüm durumlarıyla orantıları sebebiyle öne çıkan bazı veriler hakkında şunlar söylenebilir:

- HistoryHTN: Hastaların hiper tansiyonu var ise ölüm oranı yüksek.

- HistoryDiabetes: Hastanın diyabeti varsa ölüm oranı yüksek.

- AgeBaseline: Yaş 50'nin üstüne çıktıkça ölüm oranı artıyor.

- Sex : Genellikle erkek hastalarda daha çok ölüm gözlemlenmiştir.

In [ ]:
for i in dataset.columns:
    if dataset[i].nunique() < 3:
        print(i)

İkili değere sahip olan veri kolonlarını elde ettik.

In [ ]:
feature = ["Sex","HistoryDiabetes","HistoryCHD","HistoryVascular","HistorySmoking","HistoryHTN ", "HistoryDLD", "HistoryDLD","DLDmeds","DMmeds","HTNmeds","ACEIARB","EventCKD35","TIME_YEAR"]
for i in feature:
    plt.figure(figsize=(12,12))
    sns.swarmplot(x=dataset["EventCKD35"], y=dataset[i])
    sns.boxenplot(x=dataset["EventCKD35"], y=dataset[i])
    plt.show()

In [ ]:
# YAŞ DAĞILIMININ HANGİ YAŞLAR ARASINDA OLDUĞUNA BAKTIK
fig=ff.create_distplot([dataset['AgeBaseline'].values],['AgeBaseline'])
fig.update_layout(title_text='Yaş Dağılımı grafiği')
fig.show()

In [ ]:
male = dataset[dataset["Sex"]==1]
female = dataset[dataset["Sex"]==0]


labels = ['Male - Survived','Male - Not Survived', "Female -  Survived", "Female - Not Survived"]
values = [len(male[dataset["EventCKD35"]==0]),len(male[dataset["EventCKD35"]==1]),
          len(female[dataset["EventCKD35"]==0]),len(female[dataset["EventCKD35"]==1])]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(title_text="Cinsiyet Üzerinden Hayatta Kalma Analizi")
fig.show()

### Grafiğe göre ölen erkek sayısı ölen kadın sayısından fazladır.

In [ ]:
# Zamanı ölüm olayıyla karşılaştırdık. Mavi ölen kişiyi temsil ediyor ve kırmızı hayatta kalan kişiyi temsil ediyor
fig = px.histogram(dataset, x="TIME_YEAR", color="EventCKD35", marginal="box", hover_data=dataset.columns)
fig.show()

### Ölüm çoğunlukla Cinsiyet, Diabet hastalığı, Hiper tansiyon hastalığı gibi önemli özelliklerden etkilenmektedir
#### Önemli özellikleri yaş sütunu ile çizip ölüm olayıyla ilişkilendirdik

## Hayatta kalan ve ölüm sayılarını hesaplamak için ayrıntılı grafikler

In [ ]:
male = dataset[dataset["Sex"]==1]
female = dataset[dataset["Sex"]==0]


labels = ['Male - Survived','Male - Not Survived', "Female -  Survived", "Female - Not Survived"]
values = [len(male[dataset["EventCKD35"]==0]),len(male[dataset["EventCKD35"]==1]),
          len(female[dataset["EventCKD35"]==0]),len(female[dataset["EventCKD35"]==1])]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(title_text="Cinsiyet Üzerinden Hayatta Kalma Analizi")
fig.show()

## Grafiğe göre ölen ve yaşayan erkek oranı kadınlardan fazladır. Bunun sebebi veri setindeki erkek oranı olabilir.

In [ ]:
HistoryDiabetes = dataset[dataset["HistoryDiabetes"]==1]["AgeBaseline"]
not_HistoryDiabetes = dataset[dataset["HistoryDiabetes"]==0]["AgeBaseline"]
hist_data = [HistoryDiabetes,not_HistoryDiabetes]
group_labels = ['Diyabet hastası olanlar', 'Diyabet hastası olmayanlar']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.5)
fig.update_layout(title_text="Diyabet Durumunda Yaş Analizi")
fig.show()

In [ ]:
labels = ['Diyabet Hastası olmayanlar','Diyabet Hastaları']
diabetes_yes = dataset[dataset['HistoryDiabetes']==1]
diabetes_no = dataset[dataset['HistoryDiabetes']==0]
values = [len(diabetes_no), len(diabetes_yes)]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(
    title_text="Diyabet Üzerine Analiz")
fig.show()

In [ ]:

yes_diabetes=dataset[dataset['diabetes']==1]
no_diabetes=dataset[dataset['diabetes']==0]

dia_deaths=yes_diabetes[dataset['EventCKD35']==1]
dia_live=yes_diabetes[dataset['EventCKD35']==0]
no_dia_deaths=no_diabetes[dataset['EventCKD35']==1]
no_dia_live=no_diabetes[dataset['EventCKD35']==0]
labels=['Ölen Diyabet Hastaları','Yaşayan Diyabet Hastaları','Diyabet Hastası olmayan Ölenler','Diyabet Hastası Olmayan Yaşayanlar']
values=[len(dia_deaths),len(dia_live),len(no_dia_deaths),len(no_dia_live)]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(title_text='testing')
fig.show()

### Grafiğe göre diyabet hastası olanların ölme oranı daha fazla ama veri setimizde diyabet hastası olmayanların oranı daha fazla olması buna sebep olabilir.

In [ ]:
#Yinelenen değerler
df.duplicated().sum()

# VERİ ÖN İŞLEME
- Özelliklere X ve hedefe y olarak değer atama
- Özelliklerin ölçeklendirmesini gerçekleştirin
- Bölünmüş test ve eğitim setleri

In [ ]:
##özelliklere X ve hedefe y olarak değer atama
X=dataset.drop(["EventCKD35"],axis=1)
y=dataset["EventCKD35"]

### Özellik ölçeklendirme, makine öğrenimi modellerini eğitmeden önce dikkate almamız gereken en temel ön işleme adımlarından biridir.
### Şimdi, ölçekleyiciyi sığdırmak ve eğitim örneklerini gözlemledikten sonra öğrenilen parametreleri kullanarak hem eğitim hem de test setlerini dönüştürmek için verileri ölçeklendirmemiz gerekiyor

In [ ]:
#Özellikler için standart bir ölçekleyici kur
col_names = list(X.columns)
s_scaler = preprocessing.StandardScaler()
X_dataset= s_scaler.fit_transform(X)
X_dataset = pd.DataFrame(X_dataset, columns=col_names)   
X_dataset.describe().T

In [ ]:
#spliting test ve eğitim setleri
X_train, X_test, y_train,y_test = train_test_split(X_dataset,y,test_size=0.25,random_state=7)

In [ ]:
# ANN'yi başlatma
model = Sequential()
# KATMANLAR
model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
model.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 32, epochs = 20)

### Verimizi ölçeklendirdikten sonra 20 dönem boyunca eğittik.
### Ölçeklendirmeden önce yaptığımız eğitime göre daha fazla kesinlik sağlayarak %60'lara ulaştığımızı görüyoruz. 
### Ölçeklendirmeden sonra ikili çapraz entropi kaybının düştüğünü gözlemliyoruz.
### Bu gözleme göre verimizin daha düzgün eğitildiğini söyleyebiliriz.

#### Kolaylık adına, sonuçları iki boyutlu bir alanda görselleştirdik

In [ ]:
#ölçeklendirilmiş özelliklere bakma
colours =["#774571","#b398af","#f1f1f1" ,"#afcdc7", "#6daa9f"]
plt.figure(figsize=(20,10))
sns.boxenplot(data = X_dataset,palette = colours)
plt.xticks(rotation=90)
plt.show()

# Model Oluşturma

- ANN'nin başlatılması Katman ekleyerek tanımlama
- YSA'nın Derlenmesi
- YSA'nın Eğitilmesi

In [ ]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, #iyileştirme olarak sayılacak minimum değişiklik miktarı
    patience=20, # durmadan önce kaç epochs beklenmeli
    restore_best_weights=True)

# YSA'yı başlatma
model = Sequential()
# KATMANLAR
model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
model.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# YSA'nın Derlenmesi
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# YSA'nın eğitilmesi
history = model.fit(X_train, y_train, batch_size = 32, epochs = 500,callbacks=[early_stopping], validation_split=0.2)

In [ ]:
val_accuracy = np.mean(history.history['val_accuracy'])
print("\n%s: %.2f%%" % ('val_accuracy', val_accuracy*100))

# Dönemler boyunca eğitim ve doğrulama kaybını planlama

In [ ]:
#Validation accuracy = doğrulama doğruluğu
# training = eğitim
#epochs = döenmler
# Accuracy = kayıplar
history_df = pd.DataFrame(history.history)

plt.plot(history_df.loc[:, ['loss']], "#6daa9f", label='Training loss')
plt.plot(history_df.loc[:, ['val_loss']],"#774571", label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")

plt.show()

## Dönemler üzerinde eğitim ve doğrulama doğruluğunu planlama

In [ ]:
history_df = pd.DataFrame(history.history)

plt.plot(history_df.loc[:, ['accuracy']], "#6daa9f", label='Training accuracy')
plt.plot(history_df.loc[:, ['val_accuracy']], "#774571", label='Validation accuracy')

plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# SONUÇLAR

- Test seti üzerinde test Karışıklık matrisinin değerlendirdik 
- Sınıflandırma raporunun değerlendirdik

In [ ]:
#Test seti sonuçlarını tahmin etme
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
np.set_printoptions()

In [ ]:
# predicted = tahmin edilen
# actual = gerçek
pd.DataFrame(list(zip(y_test, y_pred)), columns=['Actual', 'Predicted'])

In [ ]:
print(classification_report(y_test, y_pred))